In [1]:
using JLD2
using FileIO
using LinearAlgebra
using Plots
using Dierckx
using LinearAlgebra
using LaTeXStrings
using Meshfree4ScalarEq.ScalarHyperbolicEquations
using Meshfree4ScalarEq.ParticleGrids
using Meshfree4ScalarEq.TimeIntegration
using Meshfree4ScalarEq.Interpolations
using Meshfree4ScalarEq.SimSettings
using Meshfree4ScalarEq
plotlyjs()

Plots.PlotlyJSBackend()

In [2]:
function mapGrid(x, xmin, xmax)
    temp = x .- xmin
    temp1 = mod.(temp, xmax-xmin)
    return temp1 .+ xmin
end

function smoothInit(x::Real, y::Real)
    return exp(-x^2 - y^2)
end

smoothInit (generic function with 1 method)

In [3]:
# TODO: How to compute error correctly? Compute exact solution on irregular grid?
function computeError(particleGrid::ParticleGrid2D, initFunc::Function, time::Real, vel::Tuple{Float64, Float64}; normP = 2)    
    # Flip boundary particles to other edge of domain such that the interpolating polynomial doesn't need to be extrapolated
    function flipBoundary(pos::Tuple{Float64, Float64}, particleGrid::ParticleGrid)
        if (pos[1] == particleGrid.xmin) && (pos[2] == particleGrid.ymin)
            return (particleGrid.xmax, particleGrid.ymax)
        elseif pos[1] == particleGrid.xmin
            return (particleGrid.xmax, pos[2])
        elseif pos[2] == particleGrid.ymin
            return (pos[1], particleGrid.ymax)
        else
            error("Shouldn't happen")
        end
    end

    # Interpolate solution on regular grid
    rhosReduced = collect(map(particle -> particle.rho, particleGrid.grid))
    xsIrregularReduced = collect(map(particle -> particle.pos, particleGrid.grid))
    # rhosFlipped = [particle.rho for particle in particleGrid.grid if particle.boundary]
    # xsIrregularFlipped = [flipBoundary(particle.pos, particleGrid) for particle in particleGrid.grid if particle.boundary]
    # rhos = vcat(rhosReduced, rhosFlipped)
    # xsIrregular = vcat(xsIrregularFlipped, xsIrregularReduced)
    # xIrregular = [x[1] for x in xsIrregular]
    # yIrregular = [x[2] for x in xsIrregular]
    # xRegular = [x[1] for x in xsRegular]
    # yRegular = [x[2] for x in xsRegular]

    # rhoSpline = Spline2D(xIrregular, yIrregular, rhos; kx=2, ky=2, s=100000)
    # numericalSol = evaluate(rhoSpline, xRegular, yRegular)

    # Compute exact solution
    # xsRegular = [(x, y) for x in particleGrid.xmin:particleGrid.dx/2:particleGrid.xmax for y in particleGrid.ymin:particleGrid.dy/2:particleGrid.ymax]
    exactSol = zeros(length(xsIrregularReduced))
    for (index, (x, y)) in enumerate(xsIrregularReduced)
        exactSol[index] = initFunc(mapGrid(x - vel[1]*time, particleGrid.xmin, particleGrid.xmax), mapGrid(y - vel[2]*time, particleGrid.ymin, particleGrid.ymax))
    end

    # Return absolute error
    return norm(exactSol - rhosReduced, normP)/norm(exactSol, normP)
end


computeError (generic function with 1 method)

In [12]:
folder = "$(@__DIR__)/data/";
a = (1.0, 1.0);
tmax = 2.5;
Algorithms = ["MeshfreeUpwind1" "RK3Upwind1" "RK3MOODUpwind2" "RK3MUSCL1" "RK3MUSCL2" "RK3MOODMUSCL1" "RK3MOODMUSCL2" "RK3WENO2"]
Ns = [30; 50; 70; 100]
errors = zeros((length(Ns), length(Algorithms)))
for file in readdir(folder)
    Alg, _, _, N = split(file, "_")
    # println(Alg, ", ", N)
    N = parse(Float64, N)
    if N in Ns
        # println(findfirst(x -> x == Alg, Algorithms), ", ", Alg)
        # println(findfirst(x -> x == N, Ns), ", ", N)
        algIndex = findfirst(x -> x == Alg, Algorithms)[2]
        NIndex = findfirst(x -> x == N, Ns)
        particleGrid = load(folder*file*"/data/step1.jld2")["particleGrid"]
        # println(algIndex, ", ", NIndex)
        errors[NIndex, algIndex] = computeError(particleGrid, smoothInit, tmax, a)
    end
end

BoundsError: BoundsError: attempt to access 2-element Vector{SubString{String}} at index [3]

In [5]:
bools = [true; true; true; true; true]
p1 = plot(Ns, errors[:, bools], xscale=:log10, yscale=:log10, label=Algorithms, legend=:bottomleft, ylabel="Relative error", xlabel=L"N_x", title="Global error at t=$(tmax)s", ls=:solid, markershape=:circle, markersize=3, size=(800, 600))
plot!(p1, Ns, 4 ./ (Ns.^0.5), label="Reference order 1/2", ls=:dash)
plot!(p1, Ns, 75 ./ Ns, label="Reference order 1", ls=:dash)
plot!(p1, Ns, 300 ./ (Ns.^2), label="Reference order 2", ls=:dash)
savefig(p1, "$(@__DIR__)/convergence.pdf")


┌ Warning: No strict ticks found
└ @ PlotUtils /users/cip/techno1/cac13ruw/.julia/packages/PlotUtils/8mrSm/src/ticks.jl:194
┌ Warning: attempting to remove probably stale pidfile
│   path = /net/nascip131/users/cip/techno1/cac13ruw/.jlassetregistry.lock
└ @ Pidfile /users/cip/techno1/cac13ruw/.julia/packages/Pidfile/DDu3M/src/Pidfile.jl:260
┌ Warning: No strict ticks found
└ @ PlotUtils /users/cip/techno1/cac13ruw/.julia/packages/PlotUtils/8mrSm/src/ticks.jl:194


"/scratch/cac13ruw/meshfree4scalareq/numericalExperiments/convergence2D/convergence.pdf"